<a href="https://colab.research.google.com/github/kannanv93/Facial-Video-Emotion-Recognizer/blob/master/CCM_Project_Colab_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm, tqdm_notebook
from joblib import Parallel, delayed
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from tqdm import tqdm

# for reading and displaying images

%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# torchvision for pre-trained models
from torchvision import models

In [0]:
import shutil, sys   
from google.colab.patches import cv2_imshow
import copy

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#@title
!unzip "/content/drive/My Drive/Colab Notebooks/Images/image_small.zip" -d "/content/drive/My Drive/Colab Notebooks/Images/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-14-4.05.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-14-4.2.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-14-4.35.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-15-0.15.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-15-0.3.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-15-0.45.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-15-0.6.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-15-0.75.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/images/01-02-06-01-01-01-15-0.9.jpg  
  inflating: /content/drive/My Drive/Colab Notebooks/Images/im

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Images/images")


In [0]:
len(os.listdir())

42054

In [0]:
meta_data = pd.DataFrame(columns=['file_name','modality', 'voc_channel','emotion',"emotion_label",'emot_intensity',' statement', 'rep', 'actor','sec_frame'])

In [0]:
for i in tqdm_notebook(range(len(files))):
    modality, voc_channel,emotion, emot_intensity, statement, rep, actor = [int(x) for x in files[i].split(".jpg")[0].split("-")[:7]]
    sec_frame = float(files[i].split(".jpg")[0].split("-")[7])
    
    
    
    if int(actor)%2 == 0:
        gender = "female"
    else:
        gender = "male"
        
    if emotion == 1:
        lb = "neutral"
    elif emotion == 2:
        lb = "calm"
    elif emotion == 3:
        lb = "happy"
    elif emotion == 4:
        lb = "sad"
    elif emotion == 5:
        lb = "angry"
    elif emotion == 6:
        lb = "fearful"
    elif emotion == 7:
        lb = "disgust"
    elif emotion == 8:
        lb = "surprised"
    else:
        lb = "none"
        
    meta_data.loc[i] = [str(files[i]),modality, voc_channel,emotion, lb,emot_intensity, statement, rep, actor,sec_frame]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [0]:
meta_data['split'] =  np.where((meta_data.actor ==23) | (meta_data.actor ==24), 'Test',(np.where((meta_data.actor ==21) | (meta_data.actor ==22),'Val','Train')))
#meta_data.drop(meta_data.index[meta_data['emotion_label'] == 'surprised'], inplace = True)
#meta_data.loc[meta_data.emotion_label=='calm',['emotion','emotion_label']]= 1,'neutral'

In [0]:
meta_data.to_csv("/content/drive/My Drive/Colab Notebooks/Images/meta_data.csv

In [0]:
meta_data.index = range(len(meta_data))
meta_data.shape

(40748, 11)

In [0]:
train  = meta_data[meta_data.split=="Train"]
val = meta_data[meta_data.split=='Val']
test = meta_data[meta_data.split=='Test']

In [0]:
train.index = range(len(train))
val.index = range(len(val))
test.index = range(len(test))
#del meta_data

In [0]:
!rm "/content/drive/My Drive/Colab Notebooks/Images/test/01-01-01-01-01-01-23-0.5.jpg"

In [0]:
os.listdir("/content/drive/My Drive/Colab Notebooks/Images/test/")

[]

In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/Images/images/01-01-01-01-01-01-01-0.5.jpg" "/content/drive/My Drive/Colab Notebooks/Images/test/"

In [0]:
meta_data = pd.read_csv("master_meta_data.csv")
meta_data.head()

,Unnamed: 0,file_name,modality,voc_channel,emotion,emotion_label,emot_intensity,statement,rep,actor,sec_frame,split
0,0,01-01-01-01-01-01-01-0.5.jpg,1,1,1,neutral,1,1,1,1,0.5,Train
1,1,01-01-01-01-01-01-01-0.jpg,1,1,1,neutral,1,1,1,1,0.0,Train
2,2,01-01-01-01-01-01-01-1.0.jpg,1,1,1,neutral,1,1,1,1,1.0,Train
3,3,01-01-01-01-01-01-01-1.5.jpg,1,1,1,neutral,1,1,1,1,1.5,Train
4,4,01-01-01-01-01-01-01-2.0.jpg,1,1,1,neutral,1,1,1,1,2.0,Train


In [0]:
set(meta_data.emotion_label)

{'angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised'}

In [0]:
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test"

In [0]:
os.listdir("/content/drive/My Drive/Colab Notebooks/Images/datadir/val")

['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [0]:
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/angry/"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/calm"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/disgust"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/fearful"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/happy"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/neutral"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/sad"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/train/surprised"
													   
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/angry"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/calm"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/disgust"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/fearful"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/happy"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/neutral"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/sad"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/test/surprised"
													   
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/angry"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/calm"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/disgust"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/fearful"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/happy"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/neutral"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/sad"
!mkdir "/content/drive/My Drive/Colab Notebooks/Images/datadir/val/surprised"



In [0]:
os.listdir("/content/drive/My Drive/Colab Notebooks/Images/train/")

['angry', 'calm', 'disgust', 'fearful', 'happy', 'sad', 'surprised', 'neutral']

In [0]:
os.listdir("/content/drive/My Drive/Colab Notebooks/Images/datadir/train/angry/")

[]

In [0]:
meta_data =pd.read_csv("/content/drive/My Drive/Colab Notebooks/Images/master_meta_data.csv")
meta_data.shape

(42054, 12)

In [0]:
meta_data.split.value_counts()

train    34441
val       3816
test      3797
Name: split, dtype: int64

In [0]:
emotion = meta_data.emotion[0]
split = meta_data.split[0]
if emotion == 1:
    lb = "neutral"
  
elif emotion == 2:
    lb = "calm"
  
elif emotion == 3:
    lb = "happy"
  
elif emotion == 4:
    lb = "sad"
  
elif emotion == 5:
    lb = "angry"
  
elif emotion == 6:
    lb = "fearful"
  
elif emotion == 7:
    lb = "disgust"
  
elif emotion == 8:
    lb = "surprised"
  
else:
    lb = "none"

end_loc = str("/content/drive/My Drive/Colab Notebooks/Images/datadir/" + str(split)+"/"+str(lb)+"/")
end_loc

'/content/drive/My Drive/Colab Notebooks/Images/datadir/train/neutral/'

In [0]:
for i in tqdm_notebook(range(len(meta_data))):

  emotion = meta_data.emotion[i]
  split = meta_data.split[i]
  
  if emotion == 1:
      lb = "neutral"
   
  elif emotion == 2:
      lb = "calm"
   
  elif emotion == 3:
      lb = "happy"
   
  elif emotion == 4:
      lb = "sad"
   
  elif emotion == 5:
      lb = "angry"
   
  elif emotion == 6:
      lb = "fearful"
   
  elif emotion == 7:
      lb = "disgust"
   
  elif emotion == 8:
      lb = "surprised"
   
  else:
      lb = "none"
  
  end_loc = str("/content/drive/My Drive/Colab Notebooks/Images/datadir/" + str(split)+"/"+str(lb)+"/")
  start_loc  = str("/content/drive/My Drive/Colab Notebooks/Images/images/"+meta_data.file_name[i])
  shutil.copy(start_loc,end_loc)

   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [0]:
os.listdir("/content/drive/My Drive/Colab Notebooks/Images/datadir/test/angry/")

['01-01-05-01-01-01-23-0.5.jpg',
 '01-01-05-01-01-01-23-0.jpg',
 '01-01-05-01-01-01-23-1.0.jpg',
 '01-01-05-01-01-01-23-1.5.jpg',
 '01-01-05-01-01-01-23-2.0.jpg',
 '01-01-05-01-01-01-23-2.5.jpg',
 '01-01-05-01-01-01-23-3.0.jpg',
 '01-01-05-01-01-01-24-0.5.jpg',
 '01-01-05-01-01-01-24-0.jpg',
 '01-01-05-01-01-01-24-1.0.jpg',
 '01-01-05-01-01-01-24-1.5.jpg',
 '01-01-05-01-01-01-24-2.0.jpg',
 '01-01-05-01-01-01-24-2.5.jpg',
 '01-01-05-01-01-01-24-3.0.jpg',
 '01-01-05-01-01-01-24-3.5.jpg',
 '01-01-05-01-01-01-24-4.0.jpg',
 '01-01-05-01-01-02-23-0.5.jpg',
 '01-01-05-01-01-02-23-0.jpg',
 '01-01-05-01-01-02-23-1.0.jpg',
 '01-01-05-01-01-02-23-1.5.jpg',
 '01-01-05-01-01-02-23-2.0.jpg',
 '01-01-05-01-01-02-23-2.5.jpg',
 '01-01-05-01-01-02-23-3.0.jpg',
 '01-01-05-01-01-02-24-0.5.jpg',
 '01-01-05-01-01-02-24-0.jpg',
 '01-01-05-01-01-02-24-1.0.jpg',
 '01-01-05-01-01-02-24-1.5.jpg',
 '01-01-05-01-01-02-24-2.0.jpg',
 '01-01-05-01-01-02-24-2.5.jpg',
 '01-01-05-01-01-02-24-3.0.jpg',
 '01-01-05-01-01-0